## BDS HW 3

In [58]:
!pip install requests
!pip install bs4
!pip install csv
!pip install PyPDF2
!pip install tika
!pip install tqdm
!pip install pandas
!pip install numpy

ERROR: Could not find a version that satisfies the requirement csv (from versions: none)
ERROR: No matching distribution found for csv


In [59]:
# imports
import requests 
from bs4 import BeautifulSoup
import csv
import os
import PyPDF2
from tika import parser
from tqdm import tqdm
import numpy as np
from math import log, e
import pandas as pd

In [60]:
%config Completer.use_jedi = False

In [61]:
from IPython.core.debugger import set_trace

In [62]:
# URL from which pdfs to be downloaded
url = "http://proceedings.mlr.press/v70/"
  
# Requests URL and get response object
response = requests.get(url)
  
# Parse text obtained
soup = BeautifulSoup(response.text, 'html.parser')
  
# Find all hyperlinks present on webpage
links = soup.find_all('a')
  
i = 0
  
# From all links check for pdf link and
# if present download file
# check if links already exists
directory = os.getcwd()
for link in tqdm(links):
    if ('.pdf' in link.get('href', [])):
        i += 1
  
        # Get response object for link
        response = requests.get(link.get('href'))
  
        # Write content in pdf file
        # check if pdf exists already
        if not os.path.exists(directory + "/pdf"+str(i)+".pdf"):
            pdf = open("pdf"+str(i)+".pdf", 'wb')
            pdf.write(response.content)
            pdf.close()
  
print("All PDF files downloaded")

  5%|███▋                                                                        | 57/1190 [00:05<01:44, 10.83it/s]


KeyboardInterrupt: 

In [ ]:
directory = os.getcwd()
allwords = []
for filename in tqdm(os.listdir(directory)):
    if filename.endswith(".pdf"):
        rawText = parser.from_file(filename)
        try:
            pdf_words = rawText['content'].split()
            allwords.extend([x.strip().lower() for x in pdf_words if x.strip().isalpha()])
        except AttributeError:
            print(f'There was an error with file: {filename}')

In [ ]:
# clean words in allwords
with open('words_dictionary.json') as json_file:
    eng_dict = json.load(json_file)
    allwords_clean = []
    for word in allwords:
        if word.lower() in eng_dict:
            allwords_clean.append(word)

# save all words clean
text_file = open('all _words_clean.txt', 'w')
for elem in allwords_clean:
    text_file.write(elem+'\n')
text_file.close()

In [ ]:
print(f'The reduced word list after cleaning is: {len(allwords_clean)}\nFrom the original: {len(allwords)}')

1. What are the top 10 common words in the ICML papers?

In [ ]:
# find the top 10 words
from collections import Counter

def getTop10(lst):
    counted = Counter(lst)
    return counted.most_common(10)

most_common = getTop10(allwords_clean)

In [ ]:
print(f'The most common words used are:\n {most_common}')

2. Let Z be a randomly selected word in a randomly selected ICML paper. Estimate the entropy of Z.

In [ ]:
def entropy(items, base=None):
    n_items = len(items)
    
    if n_items <= 1:
        return 0
    
    value, counts = np.unique(items, return_counts=True)
    probs = counts / n_items
    n_classes = np.count_nonzero(probs)

    if n_classes <= 1:
        return 0

    ent = 0.

    # Compute entropy
    base = e if base is None else base
    for i in probs:
        ent -= i * log(i, base)

    return ent

In [ ]:
print(entropy(allwords_clean))

3. Synthesize a random paragraph using the marginal distribution over words.

In [ ]:
# create a DF with marginal distrabution probabilities and words
counted_words = Counter(allwords_clean)
marginal_dist_words = {}
n_words = len(allwords_clean)
for key, value in counted_words.items():
    marginal_dist_words[key] = value/n_words

In [ ]:
marginal_dist_words_df = pd.DataFrame(marginal_dist_words.items(), columns=['word', 'distrabution'])

In [ ]:
sampled_words = (marginal_dist_words_df.sample(n=10, weights='distrabution')['word']).to_list()

In [ ]:
sentence = ''
for word in sampled_words:
    sentence += word+' '
sentence

Problem 3: More on Kaggle Advanced Regression.

In [63]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib

import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr
%config InlineBackend.figure_format = 'retina' #set 'png' here when working on notebook
%matplotlib inline

In [64]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [65]:
null_values = train.isnull().sum()
null_values[null_values > 0]

LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64

In [66]:
train.dtypes

Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
                  ...   
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
SalePrice          int64
Length: 81, dtype: object

In [67]:
# data pipeline
# 1. label encode categorical values
# 2. one-hot encode label encoded values
# 3. use PCA for dim reduction
# 4. XGBoost with RandomizedSearchCV to find best hyperperameters

In [68]:
# impute for categorical values and numeric values
def get_categorical_cols(df):
    cols = df.columns
    num_cols = df._get_numeric_data().columns
    return list(set(cols) - set(num_cols))

def get_numeric_cols(df):
    num_cols = df._get_numeric_data().columns
    return num_cols

def impute_data(df):
    # impute categorical data
    cat_cols = get_categorical_data(df)
    for col in cat_cols:
        df[col] = np.where(df[col].isnull(), "Unknown", df[col])
        
    # impute numeric data with interpolation
    num_cols = get_numeric_cols(df)
    for col in num_cols:
        df[col].interpolate(method='linear', direction = 'forward', inplace=True) 
        
    return df

In [69]:
imputed_df_test = impute_data(test)
imputed_df_train = impute_data(train)

In [70]:
imputed_df_train.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

Dropping columns:
1. PoolQC
2. Fence
3. MiscFeature
4. Alley
5. ID

In [71]:
def drop_cols(df):
    cols = ['PoolQC', 'Fence', 'MiscFeature', 'Alley', 'Id']
    res = df.drop(columns=cols, axis=1)
    return res

In [117]:
dropped_df_train = drop_cols(imputed_df_train)
dropped_df_test = drop_cols(imputed_df_test)

In [118]:
dropped_df_test.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'WoodDeckSF'

In [119]:
dropped_df_train.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'WoodDeckSF'

One-hot encoding categorical values

In [120]:
cat_cols_train = get_categorical_cols(dropped_df_train)
cat_cols_test = get_categorical_cols(dropped_df_test)
dum_df_train = pd.get_dummies(dropped_df_train, cat_cols_train)
dum_df_test = pd.get_dummies(dropped_df_test, cat_cols_test)

In [123]:
y_train = dum_df_train['SalePrice']
dum_df_train.drop(columns=['SalePrice'], axis=1, inplace=True)

In [129]:
aligned_train_X, aligned_test_X = dum_df_train.align(dum_df_test, join='right', axis=1)

In [130]:
len(aligned_train_X.columns)

277

In [131]:
len(aligned_test_X.columns)

277

Feature Normalization

In [132]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(aligned_train_X)
X_test = scaler.transform(aligned_test_X)

C:\Users\jloes\anaconda3\envs\BDS\lib\site-packages\sklearn\utils\extmath.py:847: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\Users\jloes\anaconda3\envs\BDS\lib\site-packages\sklearn\utils\extmath.py:689: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = op(x, *args, **kwargs)


In [139]:
test = np.isfinite(aligned_test_X).all()
test[test == False]

Series([], dtype: bool)

In [140]:
test2 = np.isfinite(aligned_train_X).all()
test2[test2 == False]

Electrical_Unknown     False
RoofStyle_Unknown      False
Exterior2nd_Unknown    False
SaleType_Unknown       False
Utilities_Unknown      False
ExterCond_Unknown      False
Exterior1st_Unknown    False
dtype: bool

In [141]:
aligned_train_X.Electrical_Unknown

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
1455   NaN
1456   NaN
1457   NaN
1458   NaN
1459   NaN
Name: Electrical_Unknown, Length: 1460, dtype: float64

In [ ]:
aligned_test_X.E